In [ ]:
!nvidia-smi

Wed Oct 19 11:53:56 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8    12W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%%capture
!pip install transformers==4.12.5 pandas torch

In [ ]:
from gensim.utils import simple_preprocess

simple_preprocess("I have a dog xx")

['have', 'dog', 'xx']

In [ ]:
import transformers

print(f"Running on transformers v{transformers.__version__}")

Running on transformers v4.12.5


## Imports

In [ ]:
import torch
import pandas as pd
import numpy as np
from pathlib import Path
from transformers import (AutoTokenizer, AutoModelForSequenceClassification,
                          PreTrainedModel, DistilBertModel, DistilBertForSequenceClassification,
                          TrainingArguments, Trainer)
from transformers.modeling_outputs import SequenceClassifierOutput

## Load data

In [ ]:
!pip install -q datasets

In [ ]:
from datasets import load_dataset
emotions = load_dataset("go_emotions", "raw")

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
emotions

DatasetDict({
    train: Dataset({
        features: ['text', 'id', 'author', 'subreddit', 'link_id', 'parent_id', 'created_utc', 'rater_id', 'example_very_unclear', 'admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral'],
        num_rows: 211225
    })
})

In [ ]:
df = emotions['train'].to_pandas()

In [ ]:
df.columns

Index(['text', 'id', 'author', 'subreddit', 'link_id', 'parent_id',
       'created_utc', 'rater_id', 'example_very_unclear', 'admiration',
       'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion',
       'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust',
       'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy',
       'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief',
       'remorse', 'sadness', 'surprise', 'neutral'],
      dtype='object')

## ..


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
from tqdm.auto import tqdm

train_df = pd.read_csv('/content/drive/MyDrive/dataset/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/dataset/test.csv')

In [ ]:
cols = ['anger', 'anticipation', 'disgust', 'fear', 'joy',
       'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust',
       'neutral']

In [ ]:
for col in cols:
    print(col,len(train_df[train_df[col] == 1]))

anger 2859
anticipation 1102
disgust 2921
fear 1363
joy 2877
love 832
optimism 2291
pessimism 895
sadness 2273
surprise 396
trust 400
neutral 218


In [ ]:
data = dict()
data['text'] = []
data['labels'] = []

for i in tqdm(range(len(train_df))):
    data['text'].append(train_df['Tweet'].iloc[i])
    tmp_list = []

    for col in cols:
        tmp_list.append(train_df[col].iloc[i])

    data['labels'].append(tmp_list)

  0%|          | 0/7724 [00:00<?, ?it/s]

In [ ]:
df = pd.DataFrame(data)

In [ ]:
len(df['text'])

7724

In [ ]:
[x|y for x,y in zip([1,0,1],[0,1,1])]

[1, 1, 1]

In [ ]:
newdata = dict()
newdata = {}
newdata['text'] = []
newdata['labels'] = []
newdata['text'] = data['text']
newdata['labels'] = data['labels']

In [ ]:
len(newdata['text'])

7724

In [ ]:
for _ in tqdm(range(40000)):
    i = np.random.randint(0,len(df))
    j = np.random.randint(0,len(df))

    if i==j:
        j = i+1

    labels = [x|y for x,y in zip(df['labels'].iloc[i],df['labels'].iloc[j])]

    newdata['labels'].append(labels)
    newdata['text'].append(df['text'].iloc[i] + ' ' + df['text'].iloc[j])


  0%|          | 0/40000 [00:00<?, ?it/s]

In [ ]:
len(newdata['text'])

105373

In [ ]:
#Go Emotion mix
for _ in tqdm(range(50000)):
    i = np.random.randint(0,len(train_df))
    j = np.random.randint(0,len(df))

    tmp_list = []
    for col in cols:
        if col not in df.columns:
            tmp_list.append(train_df[col].iloc[i])
        else:
            tmp_list.append(int(train_df[col].iloc[i]|df[col].iloc[j]))

    # labels = [x|y for x,y in zip(newdf['labels'].iloc[i],newdf['labels'].iloc[j])]

    newdata['labels'].append(labels)
    newdata['text'].append(train_df['Tweet'].iloc[i] + ' ' + df['text'].iloc[j])


  0%|          | 0/50000 [00:00<?, ?it/s]

In [ ]:
len(newdata['text'])

65373

In [ ]:
newdf = pd.DataFrame(newdata)
newdf

,text,labels
0,“Worry is a down payment on a problem you may ...,"[0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0]"
1,Whatever you decide to do make sure it makes y...,"[0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0]"
2,@Max_Kellerman it also helps that the majorit...,"[1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0]"
3,Accept the challenges so that you can literall...,"[0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0]"
4,My roommate: it's okay that we can't spell bec...,"[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
...,...,...
105368,"Yo Yo Yo,my name is #DarthVader \nI feel like ...","[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]"
105369,If a cop is going to pull a gun on you for no ...,"[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
105370,Still can't log into my fucking Snapchat #Snap...,"[1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0]"
105371,Nor hell a fury like a woman scorned -- Willia...,"[1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0]"


## Go

In [ ]:
from tqdm.auto import tqdm
import numpy as np

data = dict()
data['text'] = []
data['labels'] = []
ncount = 0

for _ in tqdm(range(100000)):
    i = np.random.randint(0,len(df))

    tmp_list = []

    for col in cols:
        if col not in df.columns:
            tmp_list.append(0)
        else:
            tmp_list.append(df[col].iloc[i])


    if 1 not in tmp_list[:-1]:
        continue

    data['text'].append(df['text'].iloc[i])
    data['labels'].append(tmp_list)

  0%|          | 0/100000 [00:00<?, ?it/s]

In [ ]:
len(data['labels']),len(data['text'])

(24062, 24062)

In [ ]:
import numpy as np

for _ in tqdm(range(400000)):
    i = np.random.randint(0,100000)
    j = np.random.randint(100000,200000)

    tmp_list = []

    for col in cols:
        if col not in df.columns:
            tmp_list.append(0)
        else:
            tmp_list.append(int(df[col].iloc[i]|df[col].iloc[j]))


    cnt = 0
    for x in tmp_list[:-1]:
        if x==1:
            cnt+=1

    if cnt<2:
        continue

    data['text'].append(df['text'].iloc[i]+' '+df['text'].iloc[j])
    data['labels'].append(tmp_list)

  0%|          | 0/400000 [00:00<?, ?it/s]

In [ ]:
len(data['labels']),len(data['text'])

(52890, 52890)

In [ ]:
for _ in tqdm(range(400000)):
    i = np.random.randint(0,50000)
    j = np.random.randint(50000,100000)
    k = np.random.randint(100000,150000)

    tmp_list = []

    for col in cols:
        if col not in df.columns:
            tmp_list.append(0)
        else:
            tmp_list.append(int(df[col].iloc[i]|df[col].iloc[j]|df[col].iloc[k]))

    cnt = 0
    for x in tmp_list[:-1]:
        if x==1:
            cnt+=1

    if cnt<3:
        continue

    data['text'].append(df['text'].iloc[i]+' '+df['text'].iloc[j] + ' ' + df['text'].iloc[k])
    data['labels'].append(tmp_list)

  0%|          | 0/400000 [00:00<?, ?it/s]

In [ ]:
len(data['labels']),len(data['text'])

(61451, 61451)

In [ ]:
for _ in tqdm(range(700000)):
    i = np.random.randint(0,50000)
    j = np.random.randint(50000,100000)
    k = np.random.randint(100000,150000)
    l = np.random.randint(150000,200000)

    tmp_list = []

    for col in cols:
        if col not in df.columns:
            tmp_list.append(0)
        else:
            tmp_list.append(int(df[col].iloc[i]|df[col].iloc[j]|df[col].iloc[k]))

    cnt = 0
    for x in tmp_list[:-1]:
        if x==1:
            cnt+=1

    if cnt<4:
        continue

    data['text'].append(df['text'].iloc[i]+' '+df['text'].iloc[j] + ' ' + df['text'].iloc[k] + ' ' + df['text'].iloc[l])
    data['labels'].append(tmp_list)

  0%|          | 0/700000 [00:00<?, ?it/s]

In [ ]:
len(data['labels']),len(data['text'])

(62660, 62660)

In [ ]:
data['text'] = data['text'][:-1500]
data['labels'] = data['labels'][:-1500]

In [ ]:


df_2 = df[df['neutral'] == 1].sample(n = 1500)
labels_neutral = []

for i in range(len(df_2)):
    tmp_list = []
    for col in cols:
        if col not in df.columns:
            tmp_list.append(0)
        else:
            tmp_list.append(df_2[col].iloc[i])

    data['text'].append(df_2['text'].iloc[i])
    data['labels'].append(tmp_list)
    labels_neutral.append(tmp_list)

# df_2.drop('labels')
df_2['labels'] = labels_neutral

df_2

In [ ]:
len(data['labels']),len(data['text'])

(64160, 64160)

In [ ]:
train_df['Tweet'] = [normalizeTweet(i) for i in train_df['Tweet_old']]
train_df

,ID,Tweet,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust,neutral,Tweet_old
0,2017-En-21441,“ Worry is a down payment on a problem you may...,0,1,0,0,0,0,1,0,0,0,1,0,“Worry is a down payment on a problem you may ...
1,2017-En-31535,Whatever you decide to do make sure it makes y...,0,0,0,0,1,1,1,0,0,0,0,0,Whatever you decide to do make sure it makes y...
2,2017-En-21068,@USER it also helps that the majority of NFL c...,1,0,1,0,1,0,1,0,0,0,0,0,@Max_Kellerman it also helps that the majorit...
3,2017-En-31436,Accept the challenges so that you can literall...,0,0,0,0,1,0,1,0,0,0,0,0,Accept the challenges so that you can literall...
4,2017-En-22195,My roommate : it 's okay that we can't spell b...,1,0,1,0,0,0,0,0,0,0,0,0,My roommate: it's okay that we can't spell bec...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7719,2018-En-01993,"@USER @USER If this did n't make me so angry ,...",1,0,1,0,0,0,0,0,0,0,0,0,@BadHombreNPS @SecretaryPerry If this didn't m...
7720,2018-En-01784,Excited to watch #stateoforigin tonight ! Come...,0,0,0,0,1,0,1,0,0,0,0,0,Excited to watch #stateoforigin tonight! Come ...
7721,2018-En-04047,"Blah blah blah Kyrie , IT , etc . @USER leavin...",1,0,1,0,0,0,0,0,1,0,0,0,"Blah blah blah Kyrie, IT, etc. @CJC9BOSS leavi..."
7722,2018-En-03041,#ThingsIveLearned The wise #shepherd never tru...,0,0,0,0,0,0,0,0,0,0,0,1,#ThingsIveLearned The wise #shepherd never tru...


In [ ]:
len(data['labels'])

7724

In [ ]:
mixed_df = pd.DataFrame(data)
mixed_df

,text,labels
0,“ Worry is a down payment on a problem you may...,"[0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0]"
1,Whatever you decide to do make sure it makes y...,"[0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0]"
2,@USER it also helps that the majority of NFL c...,"[1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0]"
3,Accept the challenges so that you can literall...,"[0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0]"
4,My roommate : it 's okay that we can't spell b...,"[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
...,...,...
7719,"@USER @USER If this did n't make me so angry ,...","[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
7720,Excited to watch #stateoforigin tonight ! Come...,"[0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0]"
7721,"Blah blah blah Kyrie , IT , etc . @USER leavin...","[1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0]"
7722,#ThingsIveLearned The wise #shepherd never tru...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]"


In [ ]:
test_df

,ID,Tweet
0,test_record_1,i could tell that my white clo this review was...
1,test_record_2,bissell big green clean machine t love love lo...
2,test_record_3,i was looking for this
3,test_record_4,bissell spot clean professional portable carpe...
4,test_record_5,bissell automate cordless rechargeable hand va...
...,...,...
395,test_record_396,fauciliedpeopledied wrong buck stops at potus ...
396,test_record_397,would go by easyjet jet wizzair b a lufthansa ...
397,test_record_398,rt over dead day zero empathy from trumpgop tr...
398,test_record_399,all offerup letgo craigslist miles ebay amazon...


In [ ]:
label_cols = cols
len(label_cols)

12

In [ ]:
id2label = {str(i):label for i, label in enumerate(label_cols)}
label2id = {label:str(i) for i, label in enumerate(label_cols)}

In [ ]:
id2label

{'0': 'anger',
 '1': 'anticipation',
 '2': 'disgust',
 '3': 'fear',
 '4': 'joy',
 '5': 'love',
 '6': 'optimism',
 '7': 'pessimism',
 '8': 'sadness',
 '9': 'surprise',
 '10': 'trust',
 '11': 'neutral'}

In [ ]:
label2id

{'anger': '0',
 'anticipation': '1',
 'disgust': '2',
 'fear': '3',
 'joy': '4',
 'love': '5',
 'optimism': '6',
 'pessimism': '7',
 'sadness': '8',
 'surprise': '9',
 'trust': '10',
 'neutral': '11'}

## Preprocess data

In [ ]:
!pip install emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 216 kB 33.2 MB/s 
  Created wheel for emoji: filename=emoji-2.1.0-py3-none-any.whl size=212392 sha256=5122ea4bc3655a1fec8b1a191175a56b9c52ce79bb3d811e3fca4459e1952036
  Stored in directory: /root/.cache/pip/wheels/77/75/99/51c2a119f4cfd3af7b49cc57e4f737bed7e40b348a85d82804
Successfully built emoji


In [ ]:
# !pip install emoji
from emoji import demojize
from nltk.tokenize import TweetTokenizer


tokenizer = TweetTokenizer()


def normalizeToken(token):
    lowercased_token = token.lower()
    if token.startswith("@"):
        return "@USER"
    elif lowercased_token.startswith("http") or lowercased_token.startswith("www"):
        return "HTTPURL"
    elif len(token) == 1:
        return demojize(token)
    else:
        if token == "’":
            return "'"
        elif token == "…":
            return "..."
        else:
            return token


def normalizeTweet(tweet):
    tokens = tokenizer.tokenize(tweet.replace("’", "'").replace("…", "..."))
    normTweet = " ".join([normalizeToken(token) for token in tokens])

    normTweet = (
        normTweet.replace("cannot ", "can not ")
        .replace("n't ", " n't ")
        .replace("n 't ", " n't ")
        .replace("ca n't", "can't")
        .replace("ai n't", "ain't")
    )
    normTweet = (
        normTweet.replace("'m ", " 'm ")
        .replace("'re ", " 're ")
        .replace("'s ", " 's ")
        .replace("'ll ", " 'll ")
        .replace("'d ", " 'd ")
        .replace("'ve ", " 've ")
    )
    normTweet = (
        normTweet.replace(" p . m .", "  p.m.")
        .replace(" p . m ", " p.m ")
        .replace(" a . m .", " a.m.")
        .replace(" a . m ", " a.m ")
    )

    return " ".join(normTweet.split())


if __name__ == "__main__":
    print(
        normalizeTweet(
            "SC has first two presumptive cases of coronavirus, DHEC confirms https://postandcourier.com/health/covid19/sc-has-first-two-presumptive-cases-of-coronavirus-dhec-confirms/article_bddfe4ae-5fd3-11ea-9ce4-5f495366cee6.html?utm_medium=social&utm_source=twitter&utm_campaign=user-share… via @postandcourier"
        )
    )

SC has first two presumptive cases of coronavirus , DHEC confirms HTTPURL ... via @USER


In [ ]:
# take sample for quick prototyping
df_sample = newdf.sample(n=1000)
df_sample.shape

(1000, 2)

In [ ]:
# create train / test splits
mask = np.random.rand(len(newdf)) < 0.8
df_train = newdf[mask]
df_val = newdf[~mask]

(df_train.shape, df_val.shape)

((84330, 2), (21043, 2))

In [ ]:
normalizeTweet('I have a dog xx @eatmycookieee #wow #fuck #sex #normal #happy ')

'I have a dog xx @USER #wow #fuck #sex #normal #happy'

## Tokenize and encode

In [ ]:
model_ckpt = "distilbert-base-uncased"
# model_ckpt = "bhadresh-savani/bert-base-go-emotion"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.12.5",
  "vocab_size": 30522
}

loading file https://huggingface.co/distilbert-base-uncased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/0e1bbfda7f63a99bb52e3915dcf10c3c92122b827d92eb2d34ce94ee79ba486c.d789d64ebf

In [ ]:
train_encodings = tokenizer(df_train["text"].values.tolist(), truncation=True)
val_encodings = tokenizer(df_val["text"].values.tolist(), truncation=True)

In [ ]:
test_encodings = tokenizer(test_df["Tweet"].values.tolist(), truncation=True)

In [ ]:
train_labels = df_train["labels"].values.tolist()
val_labels = df_val["labels"].values.tolist()

In [ ]:
class GoEmotionDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]).cuda() for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx]).cuda()
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
class TestDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]).cuda() for key, val in self.encodings.items()}
        return item

    def __len__(self):
        return 400

In [ ]:
train_dataset = GoEmotionDataset(train_encodings, train_labels)
val_dataset = GoEmotionDataset(val_encodings, val_labels)

In [ ]:
test_dataset = TestDataset(test_encodings)

In [ ]:
train_dataset[0]

{'input_ids': tensor([  101,  1523,  4737,  2003,  1037,  2091,  7909,  2006,  1037,  3291,
          2017,  2089,  2196,  2031,  1005,  1012, 11830, 11527,  1012,  1001,
         14354,  1001,  4105,  1001,  4737,   102], device='cuda:0'),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1], device='cuda:0'),
 'labels': tensor([0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0], device='cuda:0')}

In [ ]:
# sanity check
tokenizer.decode(train_dataset[0]["input_ids"])

"[CLS] “ worry is a down payment on a problem you may never have '. joyce meyer. # motivation # leadership # worry [SEP]"

## Fine-tuning

There are two ways we can implement multi-label classification:

* Creating a custom BERT model that overrides the `forward` method
* Creating a custom `Trainer` that overrides the `compute_loss` method

The second method does not work with v4.2.1 of `transformers` due to some bugs, so we'll work with the first approach instead :)

### Creating a Custom Model

In [ ]:
class DistilBertForMultilabelSequenceClassification(DistilBertForSequenceClassification):
    def __init__(self, config):
      super().__init__(config)

    def forward(self,
        input_ids=None,
        attention_mask=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        outputs = self.distilbert(input_ids.cuda(),
            attention_mask=attention_mask.cuda(),
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict)

        hidden_state = outputs[0]
        pooled_output = hidden_state[:, 0]
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output).cuda()

        loss = None
        if labels is not None:
            loss_fct = torch.nn.BCEWithLogitsLoss()
            loss = loss_fct(logits.view(-1, self.num_labels).cuda(),
                            labels.float().view(-1, self.num_labels).cuda())

        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return SequenceClassifierOutput(loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions)

In [ ]:
num_labels=12
model = DistilBertForMultilabelSequenceClassification.from_pretrained(model_ckpt, num_labels=num_labels).to('cuda')

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_10": 10,
    "LABEL_11": 11,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6,
    "LABEL_7": 7,
    "LABEL_8": 8,
    "LABEL_9": 9

In [ ]:
model.config.id2label = {
    '0': 'anger',
 '1': 'anticipation',
 '2': 'disgust',
 '3': 'fear',
 '4': 'joy',
 '5': 'love',
 '6': 'optimism',
 '7': 'pessimism',
 '8': 'sadness',
 '9': 'surprise',
 '10': 'trust',
 '11': 'neutral'
  },
model.config.label2id ={
    'anger': '0',
 'anticipation': '1',
 'disgust': '2',
 'fear': '3',
 'joy': '4',
 'love': '5',
 'optimism': '6',
 'pessimism': '7',
 'sadness': '8',
 'surprise': '9',
 'trust': '10',
 'neutral': '11'
  }
# model.config

In [ ]:
def accuracy_thresh(y_pred, y_true, thresh=0.5, sigmoid=True):
    y_pred = torch.from_numpy(y_pred)
    y_true = torch.from_numpy(y_true)
    if sigmoid:
      y_pred = y_pred.sigmoid()
    return ((y_pred>thresh)==y_true.bool()).float().mean().item()

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    return {'accuracy_thresh': accuracy_thresh(predictions, labels)}

In [ ]:
batch_size = 32
# configure logging so we see training loss
logging_steps = len(train_dataset) // batch_size

args = TrainingArguments(
    output_dir="emotion",
    evaluation_strategy = "epoch",
    save_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_steps=logging_steps
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
trainer = Trainer(
    model.cuda(),
    args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer)

In [ ]:
# sanity check that we can run evaluation
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 21043
  Batch size = 32


{'eval_loss': 0.6606487035751343,
 'eval_accuracy_thresh': 0.6635262966156006,
 'eval_runtime': 50.5513,
 'eval_samples_per_second': 416.27,
 'eval_steps_per_second': 13.016}

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 84330
  Num Epochs = 1
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 2636


Epoch,Training Loss,Validation Loss,Accuracy Thresh
1,0.385100,0.361083,0.826367


***** Running Evaluation *****
  Num examples = 21043
  Batch size = 32


Saving model checkpoint to emotion/checkpoint-2636
Configuration saved in emotion/checkpoint-2636/config.json
Model weights saved in emotion/checkpoint-2636/pytorch_model.bin
tokenizer config file saved in emotion/checkpoint-2636/tokenizer_config.json
Special tokens file saved in emotion/checkpoint-2636/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=2636, training_loss=0.38518526899163386, metrics={'train_runtime': 684.4846, 'train_samples_per_second': 123.202, 'train_steps_per_second': 3.851, 'total_flos': 1779020614701360.0, 'train_loss': 0.38518526899163386, 'epoch': 1.0})

In [ ]:
# sanity check that we can run evaluation
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 10048
  Batch size = 32


{'eval_loss': 0.28144344687461853,
 'eval_accuracy_thresh': 0.8867104649543762,
 'eval_runtime': 35.6987,
 'eval_samples_per_second': 281.467,
 'eval_steps_per_second': 8.796,
 'epoch': 2.0}

In [ ]:
# test_encodings = tokenizer(df_val["text"].values.tolist(), truncation=True)
# test_labels = df_val["labels"].values.tolist()
# val_dataset = GoEmotionDataset(val_encodings, val_labels)

In [ ]:
preds = trainer.predict(val_dataset)

***** Running Prediction *****
  Num examples = 21043
  Batch size = 32


In [ ]:
preds = list(preds)[0]
preds = 1/(1+np.exp(-preds))
preds = np.round(preds)

In [ ]:
fp = dict()

for col in cols:
    fp[col] = []

for i in range(len(preds)):
    j = 0
    for k in fp.keys():
        fp[k].append(preds[i][j])
        j += 1

In [ ]:
true_labels = dict()

for col in cols:
    true_labels[col] = []

for i in range(len(df_val['text'])):
    j = 0
    for k in true_labels.keys():
        true_labels[k].append(df_val['labels'].iloc[i][j])
        j += 1

In [ ]:
from sklearn.metrics import classification_report

for col in cols:
    print(col)
    print(classification_report(fp[col],
                                true_labels[col]))

anger
              precision    recall  f1-score   support

         0.0       0.42      0.59      0.49      3609
         1.0       0.91      0.83      0.87     17434

    accuracy                           0.79     21043
   macro avg       0.67      0.71      0.68     21043
weighted avg       0.82      0.79      0.81     21043

anticipation
              precision    recall  f1-score   support

         0.0       1.00      0.86      0.93     20930
         1.0       0.02      0.56      0.04       113

    accuracy                           0.86     21043
   macro avg       0.51      0.71      0.48     21043
weighted avg       0.99      0.86      0.92     21043

disgust
              precision    recall  f1-score   support

         0.0       0.36      0.60      0.45      2933
         1.0       0.93      0.83      0.87     18110

    accuracy                           0.80     21043
   macro avg       0.64      0.71      0.66     21043
weighted avg       0.85      0.80      0.82    

## Final_test

In [ ]:
preds = trainer.predict(test_dataset)
preds = list(preds)[0]
preds = 1/(1+np.exp(-preds))
preds = np.round(preds)

***** Running Prediction *****
  Num examples = 400
  Batch size = 32


In [ ]:
fp = dict()

for col in cols:
    fp[col] = []

for i in range(len(preds)):
    j = 0
    for k in fp.keys():
        fp[k].append(preds[i][j])
        j += 1

In [ ]:
for col in fp.keys():
    test_df[col] = np.array(fp[col]).astype(int)

test_df

,ID,Tweet,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust,neutral
0,test_record_1,i could tell that my white clo this review was...,0,0,0,0,1,0,1,0,0,0,0,0
1,test_record_2,bissell big green clean machine t love love lo...,1,0,0,0,1,1,1,0,0,0,1,0
2,test_record_3,i was looking for this,0,0,0,0,0,0,0,0,0,0,0,0
3,test_record_4,bissell spot clean professional portable carpe...,1,0,1,0,1,0,1,0,0,0,1,0
4,test_record_5,bissell automate cordless rechargeable hand va...,1,0,1,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,test_record_396,fauciliedpeopledied wrong buck stops at potus ...,1,0,1,0,0,0,1,0,0,0,0,0
396,test_record_397,would go by easyjet jet wizzair b a lufthansa ...,1,0,1,0,0,0,0,0,0,0,0,0
397,test_record_398,rt over dead day zero empathy from trumpgop tr...,1,0,1,0,1,0,0,0,0,0,1,0
398,test_record_399,all offerup letgo craigslist miles ebay amazon...,1,0,1,0,1,0,1,0,0,0,1,0


In [ ]:
test_df.to_csv("submission_12.csv")

In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/Sigmoid/model_onlySigmoid_distilBERT_norm.pt')

#BERTweet


In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer

In [ ]:
model = AutoModel.from_pretrained("vinai/bertweet-base")
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", use_fast=False)

loading configuration file https://huggingface.co/vinai/bertweet-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/356366feedcea0917e30f7f235e1e062ffc2d28138445d5672a184be756c8686.a2b6026e688d1b19cebc0981d8f3a5b1668eabfda55b2c42049d5eac0bc8cb2d
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 130,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "tokenizer_class": "BertweetTokenizer",
  "transformers_version": "4.12.5",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 64001
}

loading w

7724